# Text 2: Latent semantic indexing
**Internet Analytics - Lab 4**

---

**Group:** *Your group letter.*

**Names:**

* *Name 1*
* *Name 2*
* *Name 3*

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [20]:
import pickle as pk
import numpy as np
import scipy as sc
from scipy.sparse.linalg import svds
from lab04_helper import *
from utils import *

## Exercise 4.4: Latent semantic indexing

In [32]:
k = 300
td_matrix = load_sparse_csr("occ_matrix.npz")
# SVD decomposition
print(td_matrix.shape)
u, s, v = svds(td_matrix,k=k)

(15248, 854)


In [33]:
u_k = u[:,:k]
v_k_T = v[:k,:]
s_k = np.diag(s[:k])
print("20 biggest eigenvalues : \n",s[:20])

20 biggest eigenvalues : 
 [ 16.13695676  16.15132115  16.19533589  16.28524658  16.29884434
  16.30850426  16.35280397  16.38506679  16.42800987  16.45750665
  16.47148612  16.50836612  16.54994431  16.59233144  16.60731843
  16.62798393  16.68640329  16.77242987  16.80275446  16.81623913]


In the matrix U, each __row__ correspond to a term, and for each term the value index i correspond to how much relevant the ith concept is for this term.
In the matrix V, each __column__ correspond to a document(cours description), and for each document the value at index i correspond to how much relevant the ith concept is for this term.

## Exercise 4.5: Topic extraction

In [23]:
n = 10
m = 10
wordIndex = load_pkl("indexToWord")
coursIndex = load_pkl("indexToCourse.txt")

n_topic_relevant_term_index = np.argpartition(u_k, n,0)[-n:]
n_topic_relevant_document_index = np.argpartition(v_k_T, -n,1)[:,-n:]

for i in range(n):
    print("*************************************************")
    print("In topic",i,"the relevant terms are:")
    print(",".join(map(wordIndex.get,n_topic_relevant_term_index[:,i][:m])))
    print("An the relevant courses are:")
    print(",".join(map(coursIndex.get,n_topic_relevant_document_index[i][:m])))


*************************************************
In topic 0 the relevant terms are:
perspective,science,issues,participants,introduces,content,society,information,water,offers
An the relevant courses are:
MICRO-530,BIO-671,HUM-417(a),EE-616,MICRO-707,COM-502,ME-403,EE-490(a),MSE-463,CH-431
*************************************************
In topic 1 the relevant terms are:
information,approaches,water,transport,energy,translates,focus,policy,technology,dimensioning
An the relevant courses are:
HUM-422(a),MSE-704,ChE-402,COM-506,FIN-608,MATH-442,EE-552,BIO-105,MGT-641(d),CH-161(en)
*************************************************
In topic 2 the relevant terms are:
flexures,pneumatic,cp3,gunjan,argawal,robertson,amir,firouzeh,zhenishbek,zhakypov
An the relevant courses are:
ChE-301,FIN-406,EE-606,ME-466,COM-308,EE-470,EE-612,BIO-714,EE-519,BIO-802
*************************************************
In topic 3 the relevant terms are:
relevant,perspective,economics,institutional,introduces

## Exercise 4.6: Document similarity search in concept-space

In [41]:
wordToIndex = load_pkl("wordToIndex")
def sims(termIndex):
    return u_k[termIndex,:] @ s_k @ v_k_T
def rankAtomicTerm(atom):
    index = wordToIndex.get(atom,-1)
    print(index)
    if(index < 0):
        return np.zeros(td_matrix.shape[1])
    return sims(index)


def searchTerm(t):
    cleaned = cleaner(t)
    top = np.argsort(np.sum(list(map(rankAtomicTerm,cleaned)),0),0)
    print(top)   
searchTerm("participants")

TypeError: string indices must be integers

## Exercise 4.7: Document-document similarity